In [1]:
#https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5
#https://www.learnopencv.com/image-classification-using-transfer-learning-in-pytorch/
#https://github.com/spmallick/learnopencv/blob/master/Image-Classification-in-PyTorch/image_classification_using_transfer_learning_in_pytorch.ipynb

#Web: https://medium.com/@mohcufe/how-to-deploy-your-trained-pytorch-model-on-heroku-ff4b73085ddd

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from collections import OrderedDict
from PIL import Image
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import time

In [2]:
%sh
cp /dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER_DATA_SPLIT.ZIP /local_disk0/tmp/
unzip /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER_DATA_SPLIT.ZIP -d /local_disk0/tmp/


In [3]:
%sh
# ls /dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/
# mv /dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/*.pth /dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER_RESNET152-75EPOCHS/
# mv /dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/*.png /dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER_RESNET152-75EPOCHS/
# rm /dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/*.pth

In [4]:
image_dir = '/local_disk0/tmp/'
image_dir_train = image_dir + 'TRAIN/'
image_dir_test = image_dir + 'TEST/'
image_dir_valid = image_dir + 'VALID/'

In [5]:
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])
}

In [6]:
# Load the Data
 
# Batch size
batch_size = 32
 
# Number of classes
num_classes = len(os.listdir(image_dir_train))
 
# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=image_dir_train, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=image_dir_valid, transform=image_transforms['valid']),
    'test': datasets.ImageFolder(root=image_dir_test, transform=image_transforms['test'])
}
 
# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])
 
# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(data['test'], batch_size=batch_size, shuffle=True)
 
# Print Statistics
print('Data Set Sizes - Train: {} Valid: {}, Test: {}'.format(train_data_size, valid_data_size, test_data_size))
print('Number of Classes: {}'.format(num_classes))

Data Set Sizes - Train: 18494 Valid: 2311, Test: 2350
Number of Classes: 73

In [7]:
data['train'].class_to_idx

Out[49]: {'10108664': 0,
 '10109264': 1,
 '10109266': 2,
 '10109268': 3,
 '10109272': 4,
 '10109275': 5,
 '10118456': 6,
 '10119951': 7,
 '10119954': 8,
 '10119956': 9,
 '10119963': 10,
 '10120054': 11,
 '10121868': 12,
 '10121870': 13,
 '10121871': 14,
 '10121872': 15,
 '10121874': 16,
 '10129343': 17,
 '10129365': 18,
 '10137237': 19,
 '10137240': 20,
 '10141489': 21,
 '10141490': 22,
 '10141492': 23,
 '10141493': 24,
 '10141494': 25,
 '10142529': 26,
 '10142531': 27,
 '10142631': 28,
 '10142633': 29,
 '10142635': 30,
 '10142637': 31,
 '10142638': 32,
 '10159239': 33,
 '10172358': 34,
 '10172361': 35,
 '10172367': 36,
 '10176723': 37,
 '10176726': 38,
 '10176727': 39,
 '10176739': 40,
 '10176743': 41,
 '10176755': 42,
 '10176763': 43,
 '10176807': 44,
 '10176814': 45,
 '10191680': 46,
 '10192085': 47,
 '10192091': 48,
 '10192353': 49,
 '10192356': 50,
 '10192359': 51,
 '10192365': 52,
 '10192375': 53,
 '1019239': 54,
 '10192390': 55,
 '10193235': 56,
 '10193236': 57,
 '10193238': 58,
 '10193239': 59,
 '10193240': 60,
 '10193241': 61,
 '10193253': 62,
 '10193255': 63,
 '10193256': 64,
 '10197121': 65,
 '10197122': 66,
 '10197124': 67,
 '10197126': 68,
 '10197127': 69,
 '10197128': 70,
 '1019713': 71,
 '10202946': 72}

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = models.densenet121(pretrained=True)
model = models.resnet50(pretrained=True)
#model = models.resnet152(pretrained=True)
#model = models.inception_v3(pretrained=True)

model_name = model.__class__.__name__
print('Device: {}'.format(device))
print('ModelName: {}'.format(model_name))

Device: cuda
ModelName: ResNet

In [9]:
print(model)

ResNet(
 (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
 (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
 (layer1): Sequential(
 (0): Bottleneck(
 (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 (downsample): Sequential(
 (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 )
 )
 (1): Bottleneck(
 (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 )
 (2): Bottleneck(
 (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 )
 )
 (layer2): Sequential(
 (0): Bottleneck(
 (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 (downsample): Sequential(
 (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
 (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 )
 )
 (1): Bottleneck(
 (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 )
 (2): Bottleneck(
 (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu

In [10]:
for param in model.parameters():
    param.require_grad = False

if model_name == 'ResNet':
  fc_inputs = model.fc.in_features

if model_name == 'DenseNet':
  fc_inputs = 1024

if model_name == 'Inception3':
  fc_inputs = model.fc.in_features
  
model.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes), # Since 10 possible outputs
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)

In [11]:
optimizer = optim.Adam(model.parameters())
loss_criterion = nn.NLLLoss()

model.to(device)

##NOTE THAT DATAPARALLEL CREATES SOME CHALLENGES WHEN CONSUMING THE SAVED MODEL. NOT USING THIS FOR NOW.
#model = nn.DataParallel(model)

Out[54]: ResNet(
 (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
 (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
 (layer1): Sequential(
 (0): Bottleneck(
 (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 (downsample): Sequential(
 (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 )
 )
 (1): Bottleneck(
 (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 )
 (2): Bottleneck(
 (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 )
 )
 (layer2): Sequential(
 (0): Bottleneck(
 (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 (downsample): Sequential(
 (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
 (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 )
 )
 (1): Bottleneck(
 (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (relu): ReLU(inplace=True)
 )
 (2): Bottleneck(
 (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
 (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [12]:
def train_and_validate(model, loss_criterion, optimizer, start_epoch=1, epochs=25):
  '''
  Function to train and validate
  Parameters
      :param model: Model to train and validate
      :param loss_criterion: Loss Criterion to minimize
      :param optimizer: Optimizer for computing gradients
      :param epochs: Number of epochs (default=25)

  Returns
      model: Trained Model with best validation accuracy
      history: (dict object): Having training loss, accuracy and validation loss, accuracy
  '''

  start = time.time()
  history = []
  best_acc = 0.0

  for epoch in range(start_epoch, epochs):
      epoch_start = time.time()
      print("Epoch: {}/{}".format(epoch, epochs))

      # Set to training mode
      model.train()

      # Loss and Accuracy within the epoch
      train_loss = 0.0
      train_acc = 0.0

      valid_loss = 0.0
      valid_acc = 0.0

      for i, (inputs, labels) in enumerate(train_data_loader):

          inputs = inputs.to(device)
          labels = labels.to(device)

          # Clean existing gradients
          optimizer.zero_grad()

          # Forward pass - compute outputs on input data using the model
          outputs = model(inputs)

          # Compute loss
          loss = loss_criterion(outputs, labels)

          # Backpropagate the gradients
          loss.backward()

          # Update the parameters
          optimizer.step()

          # Compute the total loss for the batch and add it to train_loss
          train_loss += loss.item() * inputs.size(0)

          # Compute the accuracy
          ret, predictions = torch.max(outputs.data, 1)
          correct_counts = predictions.eq(labels.data.view_as(predictions))

          # Convert correct_counts to float and then compute the mean
          acc = torch.mean(correct_counts.type(torch.FloatTensor))

          # Compute total accuracy in the whole batch and add to train_acc
          train_acc += acc.item() * inputs.size(0)

          print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))


      # Validation - No gradient tracking needed
      with torch.no_grad():

          # Set to evaluation mode
          model.eval()

          # Validation loop
          for j, (inputs, labels) in enumerate(valid_data_loader):
              inputs = inputs.to(device)
              labels = labels.to(device)

              # Forward pass - compute outputs on input data using the model
              outputs = model(inputs)

              # Compute loss
              loss = loss_criterion(outputs, labels)

              # Compute the total loss for the batch and add it to valid_loss
              valid_loss += loss.item() * inputs.size(0)

              # Calculate validation accuracy
              ret, predictions = torch.max(outputs.data, 1)
              correct_counts = predictions.eq(labels.data.view_as(predictions))

              # Convert correct_counts to float and then compute the mean
              acc = torch.mean(correct_counts.type(torch.FloatTensor))

              # Compute total accuracy in the whole batch and add to valid_acc
              valid_acc += acc.item() * inputs.size(0)

              print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

      # Find average training loss and training accuracy
      avg_train_loss = train_loss/train_data_size 
      avg_train_acc = train_acc/train_data_size

      # Find average training loss and training accuracy
      avg_valid_loss = valid_loss/valid_data_size 
      avg_valid_acc = valid_acc/valid_data_size

      history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

      epoch_end = time.time()

      print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

      # Save the model
      #torch.save(model, '/dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/checkpoint-' +str(epoch) + '.pth')
      torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'class_to_idx': data['train'].class_to_idx,
      }, '/dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/checkpoint-' +str(epoch) + '.pth')

  return model, history

In [13]:
num_epochs = 20
start_epoch = 1
trained_model, history = train_and_validate(model, loss_criterion, optimizer, start_epoch, num_epochs)

torch.save(history, '/dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/history.pth')


Epoch: 1/20
Batch number: 000, Training: Loss: 4.2899, Accuracy: 0.0000
Batch number: 001, Training: Loss: 4.2997, Accuracy: 0.0312
Batch number: 002, Training: Loss: 4.3270, Accuracy: 0.0312
Batch number: 003, Training: Loss: 4.4710, Accuracy: 0.0312
Batch number: 004, Training: Loss: 4.1706, Accuracy: 0.0625
Batch number: 005, Training: Loss: 4.5966, Accuracy: 0.0000
Batch number: 006, Training: Loss: 4.0576, Accuracy: 0.0625
Batch number: 007, Training: Loss: 4.2916, Accuracy: 0.0000
Batch number: 008, Training: Loss: 3.9127, Accuracy: 0.0625
Batch number: 009, Training: Loss: 4.1740, Accuracy: 0.0625
Batch number: 010, Training: Loss: 4.0863, Accuracy: 0.0312
Batch number: 011, Training: Loss: 4.2559, Accuracy: 0.0312
Batch number: 012, Training: Loss: 3.8568, Accuracy: 0.0625
Batch number: 013, Training: Loss: 4.0844, Accuracy: 0.0000
Batch number: 014, Training: Loss: 3.8096, Accuracy: 0.0625
Batch number: 015, Training: Loss: 4.0402, Accuracy: 0.0625
Batch number: 016, Training: Loss: 4.0031, Accuracy: 0.0312
Batch number: 017, Training: Loss: 3.7987, Accuracy: 0.0625
Batch number: 018, Training: Loss: 3.7115, Accuracy: 0.1562
Batch number: 019, Training: Loss: 3.9446, Accuracy: 0.1250
Batch number: 020, Training: Loss: 3.5277, Accuracy: 0.1562
Batch number: 021, Training: Loss: 3.9451, Accuracy: 0.1250
Batch number: 022, Training: Loss: 3.6256, Accuracy: 0.0625
Batch number: 023, Training: Loss: 3.7227, Accuracy: 0.0938
Batch number: 024, Training: Loss: 3.7926, Accuracy: 0.0625
Batch number: 025, Training: Loss: 4.1130, Accuracy: 0.0938
Batch number: 026, Training: Loss: 3.8647, Accuracy: 0.0625
Batch number: 027, Training: Loss: 3.6832, Accuracy: 0.0625
Batch number: 028, Training: Loss: 3.5173, Accuracy: 0.1250
Batch number: 029, Training: Loss: 3.3986, Accuracy: 0.0938
Batch number: 030, Training: Loss: 4.1647, Accuracy: 0.0312
Batch number: 031, Training: Loss: 3.5290, Accuracy: 0.1562
Batch number: 032, Training: Loss: 3.8913, Accuracy: 0.1250
Batch number: 033, Training: Loss: 3.5447, Accuracy: 0.0938
Batch number: 034, Training: Loss: 3.4714, Accuracy: 0.1562
Batch number: 035, Training: Loss: 3.4110, Accuracy: 0.0938
Batch number: 036, Training: Loss: 3.8121, Accuracy: 0.0625
Batch number: 037, Training: Loss: 3.4884, Accuracy: 0.1875
Batch number: 038, Training: Loss: 3.6870, Accuracy: 0.0938
Batch number: 039, Training: Loss: 3.3677, Accuracy: 0.0625
Batch number: 040, Training: Loss: 3.5169, Accuracy: 0.0625
Batch number: 041, Training: Loss: 3.3415, Accuracy: 0.1875
Batch number: 042, Training: Loss: 3.8463, Accuracy: 0.0000
Batch number: 043, Training: Loss: 3.6403, Accuracy: 0.1250
Batch number: 044, Training: Loss: 3.4727, Accuracy: 0.1250
Batch number: 045, Training: Loss: 2.9434, Accuracy: 0.2188
Batch number: 046, Training: Loss: 3.4400, Accuracy: 0.0938
Batch number: 047, Training: Loss: 3.3613, Accuracy: 0.1250
Batch number: 048, Training: Loss: 3.5991, Accuracy: 0.1250
Batch number: 049, Training: Loss: 3.2089, Accuracy: 0.1250
Batch number: 050, Training: Loss: 3.3108, Accuracy: 0.2500
Batch number: 051, Training: Loss: 3.7982, Accuracy: 0.0000
Batch number: 052, Training: Loss: 3.0284, Accuracy: 0.1250
Batch number: 053, Training: Loss: 3.2295, Accuracy: 0.1562
Batch number: 054, Training: Loss: 3.3982, Accuracy: 0.1250
Batch number: 055, Training: Loss: 3.1671, Accuracy: 0.1875
Batch number: 056, Training: Loss: 3.2780, Accuracy: 0.1875
Batch number: 057, Training: Loss: 3.6020, Accuracy: 0.1250
Batch number: 058, Training: Loss: 2.9088, Accuracy: 0.1562
Batch number: 059, Training: Loss: 3.4525, Accuracy: 0.0938
Batch number: 060, Training: Loss: 3.1566, Accuracy: 0.1875
Batch number: 061, Training: Loss: 3.5877, Accuracy: 0.0938
Batch number: 062, Training: Loss: 3.9708, Accuracy: 0.1562
Batch number: 063, Training: Loss: 3.3386, Accuracy: 0.0938
Batch number: 064, Training: Loss: 3.2660, Accuracy: 0.1875
Batch number: 065, Training: Loss: 3.1489, Accuracy: 0.2188
Batch number: 066, Training:

In [14]:

model_save_path = '/dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/checkpoint-74.pth'
history_save_path = '/dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/history.pth'

#history = torch.load(history_save_path)

#OLD VERSIOn
trained_model = torch.load(model_save_path)

##NEW VERSION
#checkpoint = torch.load(PATH)
#trained_model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']


In [15]:
history = np.array(history)
plt.plot(history[:,0:2])
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0,1)
plt.savefig('/dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/model_loss_curve.png')
display(plt.show())

In [16]:
plt.plot(history[:,2:4])
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.savefig('/dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/accuracy_curve.png')
display(plt.show())

In [17]:
def computeTestSetAccuracy(model, loss_criterion):
  '''
  Function to compute the accuracy on the test set
  Parameters
      :param model: Model to test
      :param loss_criterion: Loss Criterion to minimize
  '''

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  test_acc = 0.0
  test_loss = 0.0

  # Validation - No gradient tracking needed
  with torch.no_grad():

      # Set to evaluation mode
      model.eval()

      # Validation loop
      for j, (inputs, labels) in enumerate(test_data_loader):
          inputs = inputs.to(device)
          labels = labels.to(device)

          # Forward pass - compute outputs on input data using the model
          outputs = model(inputs)

          # Compute loss
          loss = loss_criterion(outputs, labels)

          # Compute the total loss for the batch and add it to valid_loss
          test_loss += loss.item() * inputs.size(0)

          # Calculate validation accuracy
          ret, predictions = torch.max(outputs.data, 1)
          correct_counts = predictions.eq(labels.data.view_as(predictions))

          # Convert correct_counts to float and then compute the mean
          acc = torch.mean(correct_counts.type(torch.FloatTensor))

          # Compute total accuracy in the whole batch and add to valid_acc
          test_acc += acc.item() * inputs.size(0)

          print("Test Batch number: {:03d}, Test: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

  # Find average test loss and test accuracy
  avg_test_loss = test_loss/test_data_size 
  avg_test_acc = test_acc/test_data_size

  print("Test accuracy : " + str(avg_test_acc))

In [18]:
computeTestSetAccuracy(model, loss_criterion)

Test Batch number: 000, Test: Loss: 0.3229, Accuracy: 0.8750
Test Batch number: 001, Test: Loss: 0.4761, Accuracy: 0.7812
Test Batch number: 002, Test: Loss: 0.2331, Accuracy: 0.8750
Test Batch number: 003, Test: Loss: 0.4625, Accuracy: 0.8750
Test Batch number: 004, Test: Loss: 1.1806, Accuracy: 0.8438
Test Batch number: 005, Test: Loss: 0.6082, Accuracy: 0.7500
Test Batch number: 006, Test: Loss: 0.3498, Accuracy: 0.8125
Test Batch number: 007, Test: Loss: 0.4394, Accuracy: 0.7812
Test Batch number: 008, Test: Loss: 0.3101, Accuracy: 0.8438
Test Batch number: 009, Test: Loss: 0.1923, Accuracy: 0.9062
Test Batch number: 010, Test: Loss: 0.4970, Accuracy: 0.8750
Test Batch number: 011, Test: Loss: 0.2824, Accuracy: 0.8750
Test Batch number: 012, Test: Loss: 0.2427, Accuracy: 0.9375
Test Batch number: 013, Test: Loss: 0.3759, Accuracy: 0.9062
Test Batch number: 014, Test: Loss: 0.7127, Accuracy: 0.7812
Test Batch number: 015, Test: Loss: 0.4438, Accuracy: 0.7812
Test Batch number: 016, Test: Loss: 0.4426, Accuracy: 0.8438
Test Batch number: 017, Test: Loss: 0.4252, Accuracy: 0.9062
Test Batch number: 018, Test: Loss: 0.2396, Accuracy: 0.9062
Test Batch number: 019, Test: Loss: 0.3073, Accuracy: 0.8750
Test Batch number: 020, Test: Loss: 0.5928, Accuracy: 0.8438
Test Batch number: 021, Test: Loss: 0.4181, Accuracy: 0.8125
Test Batch number: 022, Test: Loss: 0.5247, Accuracy: 0.8125
Test Batch number: 023, Test: Loss: 0.9181, Accuracy: 0.6875
Test Batch number: 024, Test: Loss: 0.4494, Accuracy: 0.9062
Test Batch number: 025, Test: Loss: 0.4094, Accuracy: 0.8125
Test Batch number: 026, Test: Loss: 0.1707, Accuracy: 0.9062
Test Batch number: 027, Test: Loss: 0.4867, Accuracy: 0.8125
Test Batch number: 028, Test: Loss: 0.2976, Accuracy: 0.8750
Test Batch number: 029, Test: Loss: 0.5290, Accuracy: 0.8750
Test Batch number: 030, Test: Loss: 0.1748, Accuracy: 0.9062
Test Batch number: 031, Test: Loss: 0.1777, Accuracy: 0.9375
Test Batch number: 032, Test: Loss: 0.5118, Accuracy: 0.8438
Test Batch number: 033, Test: Loss: 0.2959, Accuracy: 0.8438
Test Batch number: 034, Test: Loss: 0.3488, Accuracy: 0.8125
Test Batch number: 035, Test: Loss: 0.5682, Accuracy: 0.8125
Test Batch number: 036, Test: Loss: 0.2419, Accuracy: 0.9375
Test Batch number: 037, Test: Loss: 0.6518, Accuracy: 0.8750
Test Batch number: 038, Test: Loss: 0.2590, Accuracy: 0.8750
Test Batch number: 039, Test: Loss: 0.5618, Accuracy: 0.7812
Test Batch number: 040, Test: Loss: 0.3224, Accuracy: 0.8750
Test Batch number: 041, Test: Loss: 0.4807, Accuracy: 0.8750
Test Batch number: 042, Test: Loss: 0.3348, Accuracy: 0.8125
Test Batch number: 043, Test: Loss: 0.2916, Accuracy: 0.8750
Test Batch number: 044, Test: Loss: 0.5562, Accuracy: 0.8125
Test Batch number: 045, Test: Loss: 0.3109, Accuracy: 0.8750
Test Batch number: 046, Test: Loss: 0.4989, Accuracy: 0.9062
Test Batch number: 047, Test: Loss: 0.3024, Accuracy: 0.9062
Test Batch number: 048, Test: Loss: 0.4859, Accuracy: 0.7812
Test Batch number: 049, Test: Loss: 0.3760, Accuracy: 0.8438
Test Batch number: 050, Test: Loss: 0.3148, Accuracy: 0.9062
Test Batch number: 051, Test: Loss: 0.3889, Accuracy: 0.8438
Test Batch number: 052, Test: Loss: 0.5145, Accuracy: 0.8438
Test Batch number: 053, Test: Loss: 0.3762, Accuracy: 0.8750
Test Batch number: 054, Test: Loss: 0.5472, Accuracy: 0.7188
Test Batch number: 055, Test: Loss: 0.3098, Accuracy: 0.9062
Test Batch number: 056, Test: Loss: 0.2262, Accuracy: 0.8438
Test Batch number: 057, Test: Loss: 0.5344, Accuracy: 0.7500
Test Batch number: 058, Test: Loss: 0.4077, Accuracy: 0.8125
Test Batch number: 059, Test: Loss: 0.5234, Accuracy: 0.7812
Test Batch number: 060, Test: Loss: 0.6588, Accuracy: 0.9062
Test Batch number: 061, Test: Loss: 0.5096, Accuracy: 0.7812
Test Batch number: 062, Test: Loss: 0.4138, Accuracy: 0.7812
Test Batch number: 063, Test: Loss: 0.2916, Accuracy: 0.9062
Test Batch number: 064, Test: Loss: 0.5050, Accuracy: 0.8125
Test Batch number: 065, Test: Loss:

In [19]:
def predict(model, test_image_name):
    '''
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_image_name: Test image

    '''
    
    transform = image_transforms['test']

    test_image = Image.open(test_image_name)
    plt.imshow(test_image)
    
    test_image_tensor = transform(test_image)

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(3, dim=1)
        for i in range(3):
            #print("Prediction", i+1, ":", idx_to_class[topclass.cpu().numpy()[0][i]], ", Score: ", topk.cpu().numpy()[0][i])
            print("Prediction", i+1, ":", topclass.cpu().numpy()[0][i], ", Score: ", topk.cpu().numpy()[0][i])

In [20]:
predict(trained_model, image_dir_valid + '1011995/133.jpg')